## comboを使うための準備

In [1]:
import pprint
sys.path.append('/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/')
pprint.pprint(sys.path)

['/Users/mkumada/Documents/class/2020/work/combo3/mywork',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles/lib/python',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/overlap_detect/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/overlap_detect/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/']


## 使用ライブラリ、関数

In [2]:
import sys
import numpy as np
import _pickle as cPickle
import scipy
import combo
import os
import urllib
import time
from combo.variable import variable
import random
import csv
from decimal import *

In [3]:
# データ取得
def load_data(path):
    A = np.loadtxt(path,skiprows=1,delimiter=',')
    if A.ndim==1:
        A = np.expand_dims(A, axis=0)
    X = A[:, :-1]
    y  = -A[:, -1]
    return X, y, A

In [14]:
    def add_data_in_file(path, data):
        with open(path, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)

In [5]:
# 濃度変換
def concentration_conversion(X):
    if X.ndim==1:
        X = X/np.sum(X)
    else:
        X = X/np.expand_dims(np.sum(X ,axis=1), axis=1)
    return X

In [6]:
# delta
def cost_func(a, b, c):
    return np.sqrt((0.2-a)**2+(0.3-b)**2+(0.5-c)**2)

## 基本方策：
		材料A, B, Cの中から１つランダルに選択し, 1ずつ添加する.
     
## 追加方策：
		目的関数との誤差がTHRESHOLD_VAL(0.5)を下回ると,
		SCALE_FACTOR(0.1)*1 添加する.

## ゴール：
		目的関数との誤差がMIN_VAL(1e-2)を下回ると探索成功.


## 最初は、ランダムにデータを入れる

In [27]:
# N種類の材料をランダムに1添加する
N = 3
TRAIN_PATH = './data/train3.csv'

select_arr = np.eye(N, dtype=float) # one hot matrix
update_quantity = select_arr[random.randrange(N)] # N種類の材料から１つをランダムに選び、1足す.
update_quantity_ratio = concentration_conversion(update_quantity) # 濃度に変換する.
delta = cost_func(update_quantity_ratio[0], update_quantity_ratio[1], update_quantity_ratio[2]) # 目指す匂いの目的関数との誤差
new_data= np.append(update_quantity, delta)

add_data_in_file(TRAIN_PATH, new_data)

## Random 探索

In [46]:
# N種類の材料をランダムに1ずつ添加する
N = 3
ITER_NUM = 50
TRAIN_PATH = './data/train3.csv'
OUT_PATH = './data/out3-1.csv'

THRESHOLD_VAL =  0.5
SCALE_FACTOR =  0.1
MIN_VAL = 1e-2

for i in range(ITER_NUM):

    X_train, t_train, _ = load_data(OUT_PATH) # read data

    if np.abs(t_train[-1]) > THRESHOLD_VAL:
        X_test = np.eye(N, dtype=float) # one hot matrix
    else:
        X_test = SCALE_FACTOR * np.eye(N, dtype=float) # one hot matrix

    update_quantity = X_train[-1] + X_test[random.randrange(N)] # N種類の材料から１つをランダムに選び、1足す.
    update_quantity_ratio = concentration_conversion(update_quantity) # 濃度に変換する.
    delta = cost_func(update_quantity_ratio[0], update_quantity_ratio[1], update_quantity_ratio[2]) # 目指す匂いの目的関数との誤差
    new_data= np.append(np.round(update_quantity, decimals=5), delta)

    add_data_in_file(OUT_PATH, new_data)

    if delta  < MIN_VAL:
        print('{}回目で収束. data:{}, delta:{}'.format(i, new_data[:-1],delta))
        break

print('{}回目の試行で収束しませんでした. data:{}, delta:{}'.format(ITER_NUM, new_data[:-1], delta))


50回目の試行で収束しませんでした. data:[3.6 2.6 2.5], delta:0.30153962612290464


##  ベイズ最適化

In [49]:
# N種類の材料からベイズ最適化で添加する
N = 3
ITER_NUM = 50
TRAIN_PATH = './data/train3.csv'
OUT_PATH = './data/out3-2.csv'
THRESHOLD_VAL =  0.5
SCALE_FACTOR =  0.1

MIN_VAL = 1e-2

for i in range(ITER_NUM):

    X_train, t_train, _ = load_data(OUT_PATH)

    if np.abs(t_train[-1]) > THRESHOLD_VAL:
        X_test = np.eye(N, dtype=float) # one hot matrix
    else:
        X_test = SCALE_FACTOR * np.eye(N, dtype=float) # one hot matrix

    X_train_ratio = concentration_conversion(X_train)

    # feature vector の各要素を正規化します。
    ave = np.mean(X_train_ratio, axis=0)
    std = np.std(X_train_ratio, axis=0)
    X_train_ratio = (X_train_ratio - ave) / (std + 1e-8)
    X_test_ratio = (X_test - ave) / (std + 1e-8)

    train_data = variable(X=X_train_ratio, t=t_train)
    test_data = variable(X=X_test_ratio)

    policy = combo.search.discrete.policy(test_X=test_data)
    # set the seed parameter
    policy.set_seed(0)
    action = policy.bayes_search_single(training=train_data, score='TS', num_rand_basis=5000)
    # score: 獲得関数(acquisition function) のタイプ。
    # TS (Thompson Sampling)
    # EI (Expected Improvement)
    # PI (Probability of Improvement)

    print(action)
    print(X_test[action])

    update_quantity = X_train[-1] + X_test[action]
    update_quantity_ratio = concentration_conversion(update_quantity) # 濃度に変換する.
    delta = cost_func(update_quantity_ratio[0], update_quantity_ratio[1], update_quantity_ratio[2]) # 目指す匂いの目的関数との誤差

    new_data = np.append(np.round(update_quantity, decimals=5), delta)

    add_data_in_file(OUT_PATH, new_data)

    if delta  < MIN_VAL:
        print('{}回目で収束. data:{}, delta:{}'.format(i, new_data[:-1],delta))
        break

print('{}回目の試行で収束しませんでした. data:{}, delta:{}'.format(ITER_NUM, new_data[:-1], delta))

likelihood -66.15431440352022
200-th epoch, marginal likelihood -67.40802619211573
250-th epoch, marginal likelihood -68.63909412545374
300-th epoch, marginal likelihood -69.85712708683901
350-th epoch, marginal likelihood -71.06676725435008
400-th epoch, marginal likelihood -72.27010245538077
450-th epoch, marginal likelihood -73.4680479516342
500-th epoch, marginal likelihood -74.66103145203363
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.]
 scale  =  1.0
 scale2 =  1.0
 

Fitting ends
Calculating score for test data
1
[0.  0.1 0. ]
Fitting starts
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood -64.71425765064346
50-th epoch, marginal likelihood -66.43049300284093
100-th epoch, marginal likelihood -67.88622751651911
150-th epoch, marginal likelihood -69.23536477911023
200-th epoch, marginal likelihood -70.540946625949
250-th epoch, marginal likelihood -71.82402823820411
300-th epoch, marginal likeli

## 課題
・１回の添加量を例えば５まで入れられるようする.

・１回の添加量で、どれかの材料を１つ選んでいるが、複数選んだ方策も選択肢に入れる.

・添加総量に制限を持たせる.
